# Tutorial on Emory ICU data

In this tutorial, we'll be exploring various files stored in a parquet format that relate to Emory's ICU patient data. Afer this tutorial you'll know:

* how to load a parquet file into a pandas dataframe
* how to use duckdb to run queries against the parquet files, without loading the data into memory
* how to plot some of the common measurements in the dataset

First, we need to setup the environment.

In [ ]:
!pip install duckdb seaborn pyarrow pandas[pyarrow]

In [ ]:
import os

import duckdb
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# set global parameters 
sns.set(font_scale=2)  # increase text size
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20

## Overview of tables in the dataset

The Emory-ICU data has the following tables:

* patients, admissions, transfers, icustays
  * tracking the patient throughout their hospital stay
* chartevents, d_items
  * charted measurements (ventilator settings, vital signs, etc)
* procedureevents
  * invasive ventilation durations
* labevents, d_labitems
  * lab measurements (chemistry, hematology, etc)
* microbiologyevents
  * microbiology measurements (blood cultures, etc)
* emar, emar_detail
  * medication administration
* diagnoses_icd, d_icd_diagnoses
  * hospital billing diagnoses
* hcpcsevents, d_hcpcs
  * billed procedures (CPT codes)

Most of the tables in MIMIC-IV are in Emory-ICU. In fact some tables are more expansive. Even though *chartevents* was an ICU only table in MIMIC-IV, it's actually hospital wide for Emory.

Nevertheless, if you are familiar with MIMIC-IV, note the following concepts (and associated tables) are not present in Emory-ICU (currently!):

* provider, caregiver
  * unique identifiers for each provider/caregiver. links to provider_id and caregiver_id.
* datetimeevents, ingredientevents, inputevents, outputevents
  * various charted data in the icu
* pharmacy
  * dispensation for medications
* prescriptions, poe, poe_detail
  * orders made by providers such as orders for medications (prescriptions), orders for x-rays, etc.
* procedures_icd, d_icd_procedures
  * hospital billed procedures

## Patient tracking

Patients are tracked through a few tables:

* patients - unique row per `subject_id`, and provides patient demographics
* admissions - unique row per `hadm_id`, and provides information about hospitalizations
* transfers - unique row per hospital transfer
* icustays - a subset of the *transfers* table, which groups together ICU stays which are contiguous

First, let's import the admissions.parquet, icustays.parquet, and patients.parquet files:

In [ ]:
patients = pd.read_parquet('/icu/emory/patients.parquet')
admissions = pd.read_parquet('/icu/emory/admissions.parquet')
transfers = pd.read_parquet('/icu/emory/transfers.parquet')
icustays = pd.read_parquet('/icu/emory/icustays.parquet')

We can take a look at rows in each table.

In [ ]:
print('=== patients ===')
display(patients.head())

print('=== icustays ===')
display(icustays.head())

We can pick a single patient and look through their stay. Let's pick 10000788.

In [ ]:
subject_id = 10000788

print('=== patients ===')
display(patients.loc[patients['subject_id']==subject_id])

print('=== admissions ===')
display(admissions.loc[admissions['subject_id']==subject_id])

print('=== transfers ===')
display(transfers.loc[transfers['subject_id']==subject_id])

print('=== icustays ===')
display(icustays.loc[icustays['subject_id']==subject_id])

# Dealing With Large Datasets

Pandas is a great tool, but it struggles with large datasets because it tries to load all the data into memory.

We can look at the size of one of the bigger tables, labevents.parquet:


In [ ]:
for table_name in ['chartevents', 'labevents', 'emar', 'emar_detail']:
    table_size = os.path.getsize(f'/icu/emory/{table_name}.parquet')
    print(f'The size of {table_name}.parquet is {table_size / 1e9:1.2f} GB')

You only have 60GB RAM shared across the team! Loading the entire dataframe will be challenging.

This is where a tool called [duckdb](https://duckdb.org/) comes in handy. This is a database engine can query data from parquet files directly without loading them into memory.

We will create aliases for each of the tables now. This does *not* load any data into memory: it creates a mapping for our software to use the data on disk.

In [ ]:
con = duckdb.connect(database=':memory:', read_only=False)

admissions = con.from_parquet('/icu/emory/admissions.parquet')
chartevents = con.from_parquet('/icu/emory/chartevents.parquet')
d_hcpcs = con.from_parquet('/icu/emory/d_hcpcs.parquet')
d_items = con.from_parquet('/icu/emory/d_items.parquet')
d_labitems = con.from_parquet('/icu/emory/d_labitems.parquet')
emar_detail = con.from_parquet('/icu/emory/emar_detail.parquet')
emar = con.from_parquet('/icu/emory/emar.parquet')
hcpcsevents = con.from_parquet('/icu/emory/hcpcsevents.parquet')
icustays = con.from_parquet('/icu/emory/icustays.parquet')
labevents = con.from_parquet('/icu/emory/labevents.parquet')
microbiologyevents = con.from_parquet('/icu/emory/microbiologyevents.parquet')
patients = con.from_parquet('/icu/emory/patients.parquet')
procedureevents = con.from_parquet('/icu/emory/procedureevents.parquet')
services = con.from_parquet('/icu/emory/services.parquet')
transfers = con.from_parquet('/icu/emory/transfers.parquet')

Now you can query all these tables as if they were in a database, and use the `.df()` call to get a dataframe out. Let's repeat our queries above, looking for a specific `subject_id`.

In [ ]:
icustays = con.sql(f"""
SELECT *
FROM icustays
WHERE subject_id = {subject_id}
""").df()
icustays.head()

This mostly shines when we start trying to do more complicated queries. For example, let's grab all the heart rates from `chartevents` and plot them for this subject.

In [ ]:
df = con.sql(f"""
SELECT subject_id, hadm_id
, c.itemid, di.label
, charttime, value, valuenum, valueuom
FROM chartevents c
INNER JOIN d_items di
ON c.itemid = di.itemid
WHERE subject_id = {subject_id}
AND c.itemid = 220045
AND valuenum IS NOT NULL
ORDER BY charttime
""").df()

plt.figure(figsize=(10,6))
sns.lineplot(x='charttime', y='valuenum', hue='label', style='label',
             data=df, markers=True, dashes=False, linewidth=2.5)

plt.xticks(rotation=45)
plt.title(f"Time-Series Plot for Subject ID: {subject_id}")
plt.show()

Keep in mind, this just scanned over 573,254,118 rows in less than a second to get this data, and did not load any of it into memory.

Looking at the plot, it looks strange. This is because the person had multiple admissions spread out over years. We should specify a single admission. Let's pick the one associated with their first ICU stay.

In [ ]:
df = con.sql(f"""
SELECT c.subject_id, c.hadm_id
, c.itemid, di.label
, charttime, value, valuenum, valueuom
FROM chartevents c
INNER JOIN (
  SELECT subject_id, intime, outtime
  FROM icustays
  WHERE subject_id = {subject_id}
  ORDER BY intime
  LIMIT 1
) i
  ON c.subject_id = i.subject_id
  AND c.charttime >= i.intime
  AND c.charttime <= i.outtime
INNER JOIN d_items di
ON c.itemid = di.itemid
WHERE c.subject_id = {subject_id}
AND c.itemid = 220045
AND valuenum IS NOT NULL
ORDER BY charttime
""").df()

plt.figure(figsize=(10,6))
sns.lineplot(x='charttime', y='valuenum', hue='label', style='label',
             data=df, markers=True, dashes=False, linewidth=2.5)

plt.xticks(rotation=45)
plt.title(f"Time-Series Plot for Subject ID: {subject_id}")
plt.show()

## What's in the dataset?

The easiest way to find out what's in the dataset is either (1) look at the columns or (2) look at the dimension tables.

Look at the columns for:

* patients, admissions, transfers, services, icustays

Look at the "d" tables for:

* chartevents -> d_items
* labevents -> d_labitems
* emar_detail -> no d table, you have to aggregate the table itself.

Aggregating the `itemid` for *chartevents* and *labevents* gives you a quick idea of what is available.

In [ ]:
chart_item_count = con.sql(f"""
SELECT c.itemid, di.label
, COUNT(c.itemid) AS n_row
FROM chartevents c
INNER JOIN d_items di
ON c.itemid = di.itemid
GROUP BY 1, 2
ORDER BY 3 DESC
""").df()

chart_item_count.head(n=10)

In [ ]:
lab_item_count = con.sql(f"""
SELECT c.itemid, di.label
, COUNT(c.itemid) AS n_row
FROM labevents c
INNER JOIN d_labitems di
ON c.itemid = di.itemid
GROUP BY 1, 2
ORDER BY 3 DESC
""").df()

lab_item_count.head(n=10)

Using these dimension tables, you can search for the measurements you are interested in (heart rate, FiO2, etc), and then write queries to get those measurements.

We can start to think about other plots.

In [ ]:
# get all charted data for a single subject
chart = con.sql(f"""
SELECT subject_id, hadm_id
, c.itemid, di.label
, charttime, value, valuenum, valueuom
FROM chartevents c
INNER JOIN d_items di
ON c.itemid = di.itemid
WHERE subject_id = 30100698
""").df()
display(chart.head())


# subselect to certain labels
labels = {'SPO2', 'PULSE', 'MAP_CUFF', 'MAP_LINE', 'GCS_TOTAL_SCORE'}

plt.figure(figsize=(12,10))
sns.lineplot(x='charttime', y='valuenum', hue='label', style='label',
             data=chart.loc[chart['label'].isin(labels)],
             markers=True, dashes=False, linewidth=2.5)

plt.xticks(rotation=45)
plt.title(f"Time-Series Plot for hadm_id")

plt.show()